In [1]:
from flask import Flask, render_template, request, redirect, url_for, session
import mysql.connector
import pandas as pd
import urllib.parse

app = Flask(__name__)
app.secret_key = '1234'  # 세션을 위한 시크릿 키 설정

def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            user='root',
            password='min990417',  # MySQL 비밀번호
            database='project',  # 사용하려는 데이터베이스 이름
            ssl_disabled=False  # SSL 연결 비활성화
        )
        if connection.is_connected():
            print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
        return connection
    except mysql.connector.Error as err:
        print(f"연결 오류: {err}")
        return None

# 사용자 데이터를 MySQL에서 불러오는 함수
def load_user_data(connection, username):
    query = f"""
    SELECT B.book_id, L.Bookname, L.Writer
    FROM BorrowedBooks B
    JOIN booklist L ON B.book_id = L.EMnumber
    JOIN Users U ON B.user_login_id = U.user_login_id
    WHERE U.username = '{username}';
    """
    user_books = pd.read_sql(query, connection)
    user_books['자료명 저자'] = user_books['Bookname'] + ' ' + user_books['Writer']
    return user_books

# 클러스터링 데이터를 불러오는 함수
def load_clustered_books():
    books_clustered = pd.read_csv('Word2Vec_GRUMHA_Agglomerative(n=35)_newnewnewtest.csv')
    return books_clustered

# 추천 도서 생성 함수
def recommend_books(connection, username, top_n=5):
    user_books = load_user_data(connection, username)
    books_clustered = load_clustered_books()

    # 클러스터 정보를 사용해 추천 도서 생성
    cluster_column_name = 'Cluster'
    combined_column_name = '자료명 저자'
    
    # 사용자가 읽은 책 데이터에 클러스터 정보 결합
    user_books_with_cluster = pd.merge(user_books, books_clustered, on=combined_column_name, how='left')
    user_clusters = user_books_with_cluster[cluster_column_name].unique()
    
    # 같은 클러스터에 속한 다른 책들 추출
    recommended_books = books_clustered[books_clustered[cluster_column_name].isin(user_clusters)]
    recommended_books = recommended_books[~recommended_books[combined_column_name].isin(user_books[combined_column_name])]
    recommended_books = recommended_books[[combined_column_name]].drop_duplicates()

    # 랜덤하게 top_n권 선택
    if len(recommended_books) > top_n:
        recommended_books = recommended_books.sample(top_n)

    # '자료명 저자'를 다시 '자료명'과 '저자'로 분리
    recommended_books[['자료명', '저자']] = recommended_books[combined_column_name].str.rsplit(n=1, expand=True)
    
    return recommended_books[['자료명', '저자']].reset_index(drop=True)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        user_login_id = request.form['user_login_id']
        password = request.form['password']

        connection = connect_to_database()
        cursor = connection.cursor(dictionary=True)
        cursor.execute("SELECT * FROM Users WHERE user_login_id = %s AND password = %s", (user_login_id, password))
        user = cursor.fetchone()
        connection.close()

        if user:
            session['username'] = user['username']  # 세션에 username 저장
            return redirect(url_for('recommend'))
        else:
            return "로그인 실패! ID 또는 비밀번호가 올바르지 않습니다."
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('username', None)
    session.pop('recommendations', None)  # 추천 도서 목록 삭제
    return redirect(url_for('login'))

@app.route('/')
def home():
    return redirect(url_for('login'))

@app.route('/recommend')
def recommend():
    if 'username' not in session:
        return redirect(url_for('login'))
    
    username = session['username']

    # 세션에 추천 도서가 없으면 새로 생성하여 저장
    if 'recommendations' not in session:
        connection = connect_to_database()
        recommendations = recommend_books(connection, username, top_n=5)
        session['recommendations'] = recommendations.to_dict(orient='records')
        connection.close()
    else:
        # 세션에서 추천 도서 불러오기
        recommendations = pd.DataFrame(session['recommendations'])

    return render_template('recommend.html', username=username, recommendations=recommendations)

@app.route('/book/<bookname_writer>')
def book_details(bookname_writer):
    connection = connect_to_database()
    cursor = connection.cursor(dictionary=True)
    
    # URL 디코딩 적용
    decoded_bookname_writer = urllib.parse.unquote_plus(bookname_writer)

    # introduction 테이블에서 해당 책의 'Book introduction'을 가져오는 쿼리
    query = "SELECT `Book introduction` FROM introduction WHERE `Bookname Writer` = %s"
    cursor.execute(query, (decoded_bookname_writer,))
    book_intro = cursor.fetchone()
    
    connection.close()
    
    if book_intro:
        return render_template('book_details.html', bookname_writer=decoded_bookname_writer, book_intro=book_intro['Book introduction'])
    else:
        return "책 정보를 찾을 수 없습니다."

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET /static/images/교통대UI.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET /static/images/배경화면2.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:35:55] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Nov/2024 14:36:00] "POST /login HTTP/1.1" 200 -


MySQL 데이터베이스에 성공적으로 연결되었습니다.


127.0.0.1 - - [12/Nov/2024 14:36:02] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:36:04] "POST /login HTTP/1.1" 302 -


MySQL 데이터베이스에 성공적으로 연결되었습니다.
MySQL 데이터베이스에 성공적으로 연결되었습니다.


/var/folders/df/c6ldw2vd4l1ffh2_6n27_xwr0000gn/T/ipykernel_72886/3335877604.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_books = pd.read_sql(query, connection)
127.0.0.1 - - [12/Nov/2024 14:36:05] "GET /recommend HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:36:05] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Nov/2024 14:36:05] "GET /static/images/교통대UI.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Nov/2024 14:36:07] "GET /book/나의%20미스터리한%20일상%20와카타케%20%25EB%2582%2598%25EB%2582%2598%25EB%25AF%25B8 HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:36:07] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Nov/2024 14:36:07] "GET /static/images/교통대UI.png HTTP/1.1" 304 -


MySQL 데이터베이스에 성공적으로 연결되었습니다.


127.0.0.1 - - [12/Nov/2024 14:36:09] "GET /recommend HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 14:36:09] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Nov/2024 14:36:09] "GET /static/images/교통대UI.png HTTP/1.1" 304 -
